# Fill in missing values using temporal NN interpolation

### Visualise results using VODCA

#### Gaps in the climatology over the selected timespan are used to seed the interpolation algorithm.

In [ ]:
import warnings
from datetime import datetime

import iris
import iris.coord_categorisation
import numpy as np
from dateutil.relativedelta import relativedelta
from tqdm.auto import tqdm

from wildfires.analysis import cube_plotting
from wildfires.data import VODCA, Datasets
from wildfires.logging_config import enable_logging
from wildfires.utils import box_mask, get_land_mask, match_shape, temporal_nn

enable_logging("jupyter")
warnings.filterwarnings("ignore", ".*converting a masked element to nan.*")
warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")

# Start and end date of the final data (inclusive).
target_timespan = (datetime(2010, 1, 1), datetime(2015, 4, 1))

vodca = Datasets(VODCA()).select_variables("VOD Ku-band")

source = vodca.dataset.copy_cubes_no_data()
source.limit_months(*target_timespan)
if not source.cube.coords("month_number"):
    iris.coord_categorisation.add_month_number(source.cube, "time")

source_masks = {}
for month_number in range(1, 13):
    single_months = source.cube.extract(iris.Constraint(month_number=month_number))

    raw_mask = single_months.data.mask

    # Add the land mask.
    land_mask = get_land_mask(n_lon=vodca.cubes[0].shape[-1])
    raw_mask &= match_shape(land_mask, raw_mask.shape)

    # Ignore regions south of -60° S.
    raw_mask &= match_shape(
        box_mask(lats=(-60, 90), lons=(-180, 180), n_lon=vodca.cubes[0].shape[-1]),
        raw_mask.shape,
    )

    source_masks[month_number] = np.mean(raw_mask, axis=0)
    _ = cube_plotting(
        source_masks[month_number], title=f"Source Mask Month {month_number}"
    )

In [ ]:
# Number of months allowed to look forward or backward for valid samples.
n_months = 3
interpolated = temporal_nn(vodca.dataset, target_timespan, n_months, verbose=1)

In [ ]:
cube_plotting(interpolated.cube[1])

In [ ]:
for month_number in range(1, 13):
    single_months = interpolated.cube.extract(
        iris.Constraint(month_number=month_number)
    )

    raw_mask = single_months.data.mask

    # Add the land mask.
    land_mask = get_land_mask(n_lon=vodca.cubes[0].shape[-1])
    raw_mask &= match_shape(land_mask, raw_mask.shape)

    # Ignore regions south of -60° S.
    raw_mask &= match_shape(
        box_mask(lats=(-60, 90), lons=(-180, 180), n_lon=vodca.cubes[0].shape[-1]),
        raw_mask.shape,
    )
    _ = cube_plotting(
        np.isclose(np.mean(raw_mask, axis=0), 1),
        title=f"Interpolated Mask Month {month_number}",
    )